Imports

In [1]:
import SimpleITK as sitk
import pandas as pd
from typing import Tuple, Dict, Any
from tqdm import tqdm
import numpy as np
import cv2
import os

Basic parameters

In [2]:
dataset_path = "datasets/Train"
label_file = "labels.csv"
dataset_name = "abus23_25"

output_folder = f"datasets/{dataset_name}_png"
slice_min_lesion_px = 25

use_classes = False  #If false, only one class is used (0)
val_frac = 0.2 # Validation fraction of the data

Load dataset info

In [3]:

def LoadAbus23(dataset_path, label_file):
    dataset = pd.read_csv(os.path.join(dataset_path, label_file))
    print("Dataset columns:", dataset.columns)
    return dataset

In [4]:
# Load dataset
dataset = LoadAbus23(dataset_path, label_file)


# Get class index
if 'label' in dataset.columns:
    classes = dataset['label'].unique().tolist()
    print("Dataset classes:", classes)
else:
    print("No classes found!")
    classes = []

Dataset columns: Index(['case_id', 'label', 'data_path', 'mask_path'], dtype='object')
Dataset classes: ['M', 'B']


Define function for reading NRRD files

In [5]:
def ReadDCM(filename: str) -> Tuple[sitk.Image, Dict[str, Any]]:
    reader = sitk.ImageFileReader()
    reader.SetFileName(filename)
    reader.LoadPrivateTagsOn()
    reader.ReadImageInformation()

    image = reader.Execute()
    metadata = {}
    for key in reader.GetMetaDataKeys():
        if reader.HasMetaDataKey(key):
            metadata[key] = reader.GetMetaData(key)     
            
    return image, metadata

Define function for 8-bits normalization

In [6]:
def normalize_8bits(image: np.ndarray):
    return (255.0 *(image - image.min()) / (image.max() - image.min())).astype(np.uint8)

### Create 8-bit slices and masks

In [7]:
images_folder = os.path.join(output_folder, "images")
masks_folder = os.path.join(output_folder, "masks")

In [8]:
os.makedirs(masks_folder, exist_ok=True)
os.makedirs(images_folder, exist_ok=True)


# Create slice from data
for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
    
    # Load data and GT
    id = row.case_id
    data, metadata = ReadDCM(os.path.join(dataset_path, row.data_path.replace('\\','/')))
    mask, _ = ReadDCM(os.path.join(dataset_path, row.mask_path.replace('\\','/')))
    
    # Check image and mask size and get numpy arrays
    assert data.GetSize() == mask.GetSize()
    data_array = sitk.GetArrayFromImage(data)
    mask_array = sitk.GetArrayFromImage(mask)
    image_size = data_array.shape[2], data_array.shape[1]
    

    # Chech the labels
    if len(classes) > 0:
        label = row.label
        assert label in classes
    else:
        label = 0
    
    # For each slice
    for idx in range(len(data_array)): #first dimension is z in numpy (z,y,x)
        data_slice = data_array[idx, ...]
        mask_slice = mask_array[idx, ...]
         
        # Use only slices with mask data
        if np.sum(mask_slice) == 0:
            continue
        
        # Binarize mask_Slice
        mask_slice = (mask_slice > 0)
        if np.sum(mask_slice) < slice_min_lesion_px:
            continue
        
        mask_slice = mask_slice.astype(np.uint8) * 255
        
        # Normalize to 8-bits
        data_slice = normalize_8bits(data_slice)
        
        
        contours, _ = cv2.findContours(mask_slice, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_list = [contour.squeeze().tolist() for contour in contours]
        
        image_name = f"{id:0>3}_{idx+1:0>3}"
        
        # Save image
        image_out = os.path.join(images_folder, f"{image_name}_{label}.png")
        cv2.imwrite(image_out, data_slice)
    
        # Save mask
        image_out = os.path.join(masks_folder, f"{image_name}_{label}.png")
        cv2.imwrite(image_out,  mask_slice)


100%|██████████| 100/100 [05:23<00:00,  3.23s/it]


### Create YOLO data

Format YOLOv8 segement:
- https://docs.ultralytics.com/datasets/segment/
- `<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>`
- Others: https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#13-prepare-dataset-for-yolov5

Format YOLOv8 detect:
- https://docs.ultralytics.com/datasets/detect/
- `<object-class> <x> <y> <width> <height>`

In [9]:
segmentation_data_path = os.path.join(output_folder, f"yolo_seg_data")
detection_data_path = os.path.join(output_folder, f"yolo_det_data")

In [10]:
import shutil

# Create folders
os.makedirs(segmentation_data_path, exist_ok=True)
os.makedirs(detection_data_path, exist_ok=True)
        
# List of image cases
list_cases = os.listdir(images_folder)

# Get classes
if not use_classes or any(len(item.split('_')) != 3 for item in list_cases):
    use_classes = False
    print("Classes are not used")
else:
    classes = sorted(set([ item[:-4].split('_')[-1] for item in list_cases]))
    print(classes)
    
# For each image
for image in tqdm(list_cases):
    
    # Check image name format
    assert len(image[:-4].split("_")) == 3
    
    # Get class label
    if use_classes:
        label = image[:-4].split("_")[-1]
        label_index = classes.index(label)
    else:
        label_index = 0
    
    # Read mask
    mask = cv2.imread(os.path.join(masks_folder, image), cv2.IMREAD_GRAYSCALE)
    mask_size = mask.shape[1], mask.shape[0]
    
    # Get contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_list = [contour.squeeze().tolist() for contour in contours]
    
    if len(contours_list) > 1:
        print(f"More than 1 coutour found for image {image}")

    # Copy image files
    shutil.copyfile(os.path.join(images_folder, image), os.path.join(detection_data_path, image))
    shutil.copyfile(os.path.join(images_folder, image), os.path.join(segmentation_data_path, image))

    # Oepn label files
    det_fp = open(os.path.join(detection_data_path, f"{image[:-4]}.txt"), "w")
    seg_fp = open(os.path.join(segmentation_data_path, f"{image[:-4]}.txt"), "w")
        
    # For each countour
    for contour in contours_list:
        
        # Get lesion countour data
        x, y = [point[0] for point in contour], [point[1] for point in contour] #TODO: use numpy
        x1, x2, y1, y2 = min(x), max(x), min(y), max(y)
        center_x, center_y = (x1+x2)/(2*mask_size[0]), (y1+y2)/(2*mask_size[1])
        width, height = (x2-x1)/mask_size[0], (y2-y1)/mask_size[1]
        
        # Check lesion size
        if width < 0.001 or height < 0.001:
            print(f"Small lesion found in image ({image})")
            continue
        
        # Write labels
        det_str = f"{center_x:0.6f} {center_y:0.6f} {width:0.6f} {height:0.6f}"
        det_fp.write(f"{label_index} {det_str}\n")   
        seg_str = " ".join([f"{point[0]/mask_size[0]:0.6f} {point[1]/mask_size[1]:0.6f}" for point in contour])
        seg_fp.write(f"{label_index} {seg_str}\n")
        
    # Close files
    det_fp.close()
    seg_fp.close()


Classes are not used


  0%|          | 11/3170 [00:00<00:30, 105.18it/s]

More than 1 coutour found for image 094_248_M.png
More than 1 coutour found for image 093_193_M.png


  1%|          | 30/3170 [00:00<00:20, 150.34it/s]

More than 1 coutour found for image 007_274_B.png
More than 1 coutour found for image 084_215_B.png


...

More than 1 coutour found for image 046_176_B.png
More than 1 coutour found for image 022_260_M.png
More than 1 coutour found for image 079_137_B.png


 99%|█████████▊| 3130/3170 [00:16<00:00, 180.74it/s]

More than 1 coutour found for image 057_198_B.png
More than 1 coutour found for image 093_232_M.png
More than 1 coutour found for image 019_202_M.png
More than 1 coutour found for image 094_244_M.png
More than 1 coutour found for image 022_311_M.png
More than 1 coutour found for image 012_202_M.png


100%|██████████| 3170/3170 [00:16<00:00, 188.00it/s]

More than 1 coutour found for image 017_212_B.png
More than 1 coutour found for image 089_182_M.png


### Create YOLO Train/Val split data

In [11]:
# Get patients list
list_items = [item[:-4] for item in os.listdir(detection_data_path) if item.endswith(".png")]
patients = list(set([ item.split("_")[0] for item in list_items]))

# Get lesion type per patient
if use_classes:
    patients_type = {}     
    for item in list_items:
        patient = item.split("_")[0] 
        label = item.split("_")[-1]
        if label not in patients_type:
            patients_type[label] = [patient]
        elif patient not in patients_type[label]:
            patients_type[label].append(patient)
else:
    patients_type = {0: patients}
    
print({k: len(v) for k, v in patients_type.items()})
    
# Distribute patients in Train/Val using val_frac with balanced lesion types
train_p = []
val_p = []
for k in patients_type.keys():
    num_train = int(len(patients_type[k])*(1-val_frac))
    train_type_p, val_type_p = patients_type[k][:num_train], patients_type[k][num_train:]
    train_p += train_type_p
    val_p += val_type_p
    
print(train_p, len(train_p))
print(val_p, len(val_p))
    
# Get distributed patient images for train and val
train = [ f"{item}.png" for item in list_items if item.split("_")[0] in train_p]
val = [ f"{item}.png" for item in list_items if item.split("_")[0] in val_p]

import random
random.seed(0)
random.shuffle(train)
random.shuffle(val)

# Generate train.txt file
train_det = os.path.join(output_folder, f"train_det.txt" ) 
with open(train_det, "w") as fp:
    fp.writelines([os.path.join(detection_data_path, t) + '\n' for t in train])
    
train_seg = os.path.join(output_folder, f"train_seg.txt" ) 
with open(train_seg, "w") as fp:
    fp.writelines([os.path.join(segmentation_data_path, t) + '\n' for t in train])
    
# Generate val.txt file
val_det = os.path.join(output_folder, f"val_det.txt")
with open(val_det, "w") as fp:
    fp.writelines([os.path.join(detection_data_path, v) + '\n' for v in val])
    
val_seg = os.path.join(output_folder, f"val_seg.txt")
with open(val_seg, "w") as fp:
    fp.writelines([os.path.join(segmentation_data_path, v) + '\n' for v in val])
    

{0: 100}
['064', '040', '081', '077', '055', '093', '054', '033', '083', '018', '051', '068', '036', '084', '065', '030', '058', '000', '004', '037', '002', '060', '046', '078', '088', '057', '052', '034', '008', '003', '095', '011', '020', '023', '049', '059', '076', '098', '028', '086', '048', '006', '045', '026', '001', '073', '017', '056', '024', '032', '085', '019', '094', '005', '069', '071', '063', '035', '042', '043', '029', '047', '013', '079', '041', '027', '015', '012', '021', '070', '050', '097', '096', '092', '072', '016', '091', '082', '044', '038'] 80
['022', '061', '066', '080', '039', '014', '009', '075', '099', '074', '007', '031', '089', '053', '062', '025', '067', '090', '010', '087'] 20
